In [1]:
import sys

import pandas as pd
import numpy as np
#import matplotlib as plt
#from matplotlib import pyplot
#import scipy as sp
#from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import CategoricalNB

from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score,recall_score,precision_score, f1_score

from sklearn.multioutput import MultiOutputClassifier

In [2]:
filename = 'events.csv'
events = pd.read_csv('./data/' + filename)
ginf = pd.read_csv('./data/' + 'ginf.csv')

In [3]:
goals = events[(events.is_goal == 1)]
moments = goals.merge(ginf, how="left").groupby("id_odsp").first()[['time','event_team','opponent','side','odd_h','odd_a']];
del goals
del events
del ginf
moments

,time,event_team,opponent,side,odd_h,odd_a
id_odsp,,,,,,
004f4ING/,82,Swansea,Southampton,2,1.68,6.50
00LMl81F/,17,AC Milan,AS Roma,2,3.15,2.60
00QH2XdM/,14,Stade de Reims,Evian Thonon Gaillard,1,2.25,4.25
00QL4t1L/,77,Borussia Dortmund,FC Ingolstadt 04,1,1.30,14.53
00WAhrVe/,28,Hamburg SV,Borussia Monchengladbach,2,1.82,5.30
...,...,...,...,...,...,...
zyEQMq2D/,18,Sevilla,Real Sociedad,2,2.26,3.65
zyJsFDYG/,44,Hannover 96,VfL Wolfsburg,1,2.44,3.20
zyKwAQxf/,31,Celta Vigo,Malaga,1,1.68,6.00


In [4]:
col = 'time'
conditions = [moments[col] < 25 , moments[col] <=45 , moments[col] <=60 , moments[col] > 60]
choices = [1 , 2 , 3, 4]
moments['time_moment'] = np.select(conditions,choices,default=np.nan)
moments

,time,event_team,opponent,side,odd_h,odd_a,time_moment
id_odsp,,,,,,,
004f4ING/,82,Swansea,Southampton,2,1.68,6.50,4.0
00LMl81F/,17,AC Milan,AS Roma,2,3.15,2.60,1.0
00QH2XdM/,14,Stade de Reims,Evian Thonon Gaillard,1,2.25,4.25,1.0
00QL4t1L/,77,Borussia Dortmund,FC Ingolstadt 04,1,1.30,14.53,4.0
00WAhrVe/,28,Hamburg SV,Borussia Monchengladbach,2,1.82,5.30,2.0
...,...,...,...,...,...,...,...
zyEQMq2D/,18,Sevilla,Real Sociedad,2,2.26,3.65,1.0
zyJsFDYG/,44,Hannover 96,VfL Wolfsburg,1,2.44,3.20,2.0
zyKwAQxf/,31,Celta Vigo,Malaga,1,1.68,6.00,2.0


In [5]:

data = pd.DataFrame();
data['time_moment'] = 0
for i, j in moments.iterrows(): 
    if j['event_team'] not in data.columns:
            data[j['event_team']] = 0
    if j['opponent'] not in data.columns:
        data[j['opponent']] = 0
    if j['side'] == 1:
        data = data.append({'time_moment': j['time_moment'], j['event_team'] : 1 , j['opponent']: -1,'odd_h' : j['odd_h'] ,'odd_a' : j['odd_a'] }, ignore_index=True)
    else:
        data = data.append({'time_moment': j['time_moment'], j['opponent'] : 1 , j['event_team']: -1,'odd_h' : j['odd_h'] ,'odd_a' : j['odd_a'] }, ignore_index=True)

data = data.replace(np.nan, 0)
data.head()
del moments

In [6]:
X = data.iloc[:,1:]
y = data.iloc[:,:1]
y = np.ravel(y)
print(X.shape)
print(y.shape)
del data
y

(8360, 144)
(8360,)


array([4., 1., 1., ..., 2., 1., 3.])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=1, stratify=y)

Unfornatanly the dataset was to big, and our system could not allocate enough memory for the Support Vector Machine Classifier, so we did not took into account this classifier.

In [10]:
def checkClf(name,clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Model name: " + name)
    #print("R2 score: {:.4f}".format(r2_score(y_test, y_pred)))
    #print("Mean square error: {:.4f}".format(mean_squared_error(y_test, y_pred)))
    #print('Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='micro')))
    #print('Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='micro')))
    #print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
    auc_roc = roc_auc_score(y_test, y_pred[:,1])
    print('Our MLP classifier obtains an AUC-ROC of {:.4f}.'.format(auc_roc))
    print('F-measure: {:.4f}'.format(f1_score(y_test, y_pred, average='micro')))
    print('The accuracy of classifying is {:.3f} %'.format(clf.score(X_test, y_test)*100))
    print('Confusion Matrix:')
    print(confusion_matrix(y_test,y_pred))
    print('Report:')
    print(classification_report(y_test,y_pred))
    print("\n")

checkClf('KNN',KNeighborsClassifier())
checkClf('SVC' , svm.SVC(kernel='linear'))
checkClf('RFT' , RandomForestClassifier(n_estimators=101))
checkClf('MLP' , MLPClassifier(max_iter = 4000,solver='sgd'))

Model name: KNN
F-measure: 0.4136
The accuracy of classifying is 41.358 %
Confusion Matrix:
[[2338  664  102   94]
 [1284  366   77   87]
 [ 525  159   28   42]
 [ 690  168   30   34]]
Report:
              precision    recall  f1-score   support

         1.0       0.48      0.73      0.58      3198
         2.0       0.27      0.20      0.23      1814
         3.0       0.12      0.04      0.06       754
         4.0       0.13      0.04      0.06       922

    accuracy                           0.41      6688
   macro avg       0.25      0.25      0.23      6688
weighted avg       0.34      0.41      0.36      6688



Model name: SVC
F-measure: 0.4629
The accuracy of classifying is 46.292 %
Confusion Matrix:
[[3008  156   10   24]
 [1709   79    7   19]
 [ 718   32    1    3]
 [ 875   36    3    8]]
Report:
              precision    recall  f1-score   support

         1.0       0.48      0.94      0.63      3198
         2.0       0.26      0.04      0.07      1814
         3.0  

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

#definition of useful methods
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 15))


title = "Learning Curves (GaussianProcess)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = GaussianProcessClassifier(max_iter_predict = 50, n_restarts_optimizer = 0, warm_start = True)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.3, 0.8),
                    cv=cv, n_jobs=4)

title = "Learning Curves (SVM, linear kernel)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = svm.SVC(gamma = 'scale', kernel = 'rbf')
plot_learning_curve(estimator, title, X, y, axes=axes[:, 1], ylim=(0.3, 0.8),
                    cv=cv, n_jobs=4)

plt.show()